# **Introduction:**

This file utilizes FISV3 to generate data that will be used to train an 
adaptive neuro-fuzzy inference system (ANFIS).

This process involves generating data randomly within the universe of 
discourse of each variable, and using this generated data to infer the 
suitability of a given robot. This is appended to a dataframe and exported
as a CSV.


# **Import Packages:**

Need to import the following packages:

In [1]:
import pandas as pd
import numpy as np
from PythonFISFunctionV3 import *
import os
from sklearn.model_selection import train_test_split

# **Data Generation Settings:**

Define the following parameters for data generation, such as the number of iterations required, as well as the max value for each input variables' universe of discourse. 

The Pandas DataFrame and FIS rulebase are also instantiated here. 

In [2]:
iterations = 15000  # max iterations
max_ud_load = 10    # max value for the load history universe of discourse
max_ud_dtt = 25     # max value for the distance to task universe of discourse
max_ud_tdt = 50     # max value for the total distance travelled universe of discourse

# instantiate rulebase:
rulebase = fis_create()

# instantiate dataframe:
columns = ['Load History', 'Distance to Task', 'Total Distance Travelled', 'Suitability']
df = pd.DataFrame(np.zeros((iterations, len(columns))), columns = columns)

# **Data Generation Loop:**

This is the main data generation loop:

In [3]:
for i in range(iterations):
    # randomly generate three robot parameters:
    load = np.random.randint(0, max_ud_load + 1)
    distance = np.random.randint(0, max_ud_dtt + 1)
    travelled = np.random.randint(0, max_ud_tdt + 1)

    # calculate suitability:
    suit = fis_solve(rulebase, load, distance, travelled)

    # create next row, append to df:
    next_row = [load, distance, travelled, suit]
    df.iloc[i] = next_row
    print(f"iteration {i+1}/{iterations}", end='\r')

# **Data Manipulation and Export:**

In this section, the data is pre-split into seperate columns, and exported. It is split here into training and testing values that can be used within MATLAB, but the entire DataFrame is also exported such that it can be split into the desired train/test splits for later use in Python.

In [4]:
# export entire dataframe as a CSV:
cwd = os.getcwd()
df.to_csv(cwd + '/Data/V3_Data.csv', index = False)

# split into train and test dataframes:
x_data = df.drop(columns = 'Suitability')
y_data = df['Suitability']
input_train, input_val_test, output_train, output_val_test = train_test_split(x_data, y_data, test_size = 0.2)
input_val, input_test, output_val, output_test = train_test_split(input_val_test, output_val_test, test_size = 0.5)


print(f"training x values: {input_train.shape}, training y values: {output_train.shape}")
print(f"validation x values: {input_val.shape}, validation y values: {output_val.shape}")
print(f"testing x values: {input_test.shape}, training y values: {output_test.shape}")

# export these as CSVs:
input_train.to_csv(cwd + '/Data/train_input.csv', index = False)
output_train.to_csv(cwd + '/Data/train_output.csv', index = False)
input_val.to_csv(cwd + '/Data/val_input.csv', index = False)
output_val.to_csv(cwd + '/Data/val_output.csv', index = False)
input_test.to_csv(cwd + '/Data/test_input.csv', index = False)
output_test.to_csv(cwd + '/Data/test_output.csv', index = False)

training x values: (12000, 3), training y values: (12000,)
validation x values: (1500, 3), validation y values: (1500,)
testing x values: (1500, 3), training y values: (1500,)
